In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'ece697ls/assignments/assignment3/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content

# Pruning

After learning, neural networks have modified and learned a set of parameters to perform our classification task. However, such parameters are costly to maintain and do not hold the same importance.

Wouldn't it be great could optimize our resource usage by dropping less important values ? This is where pruning comes into play.

Pruning is a technique that cuts off parameters/structures from a model to increase sparcity and decrease overall model size, similar to cutting leafs or branches from bushes and trees. This process can lead to smaller memory consumption with minimal accuracy reduction. Moreover, pruning the network may also provide a speedup since there will be less operations being performed.

The pruning process can be performed during the end of an epoch of training or after training is complete. Experimenting to find out which way works the best is part of the fun !

In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn

In [2]:
from ece697ls.pruning_helper import get_trained_model, test_model, load_model
from ece697ls.data_utils import get_CINIC10_data
import os

ImportError: cannot import name 'get_trained_model' from 'ece697ls.pruning_helper' (/Users/hamood/coding/ECE 562/assignment1_colab/assignment1/ece697ls/pruning_helper.py)

Below we will load a pre-trained model for you to work on. If you prefer, you can save your own model from the previous Tensorflow/Pytorch task and load it here.

In [ ]:
#This code may take a while to execute as it is training a network form scratch

data = get_CINIC10_data()
mode = 'torch'#torch or tensorflow

test_data = [data['X_test'],data['y_test']]

path = os.path.join('/content/drive/My Drive/{}'.format(FOLDERNAME), f"ece697ls/models/{mode}.model")

model = load_model(path,mode=mode)
test_model(model,test_data,mode=mode)

## Unstructured Pruning

Unstructured Pruning is usually related to the pruning of weights in neural networks. The general idea is to select a set of weights according to a policy and setting them up to zero. 

Common policies are random weight selection or selecting the smallers weights. 
Unstructured Pruning can be performed in one or multiple layers within the same network.

Altough in theory Unstructured Pruning should decrease the number of operations performed during execution there should be explicit support within the framework or hardware to bypass such operations, otherwise it will just operated over zero.

### Perform Pruning

Using the model trained in the previous step using pytorch, perform unstructured pruning in the weights of the model by removing x% of the smallest weights. 

*   Increment global pruning by 10% until reaching total of 80% pruned weights
*   Perform inference at the end of each pruning and observe the impact into the accuracy.


Note: The percentages are related to the entire model, not per layer.



In [ ]:
################################################################################
# TODO: Perform unstructured Pruning over the trained model using 3 different  
# prunning percentages.                                
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

import torch
import torch.nn.utils.prune as prune

print("Original model accuracy:")
test_model(model, test_data, mode=mode)
print("\n" + "="*60 + "\n")

# Get all parameters to prune (weights from Linear and Conv2d layers)
parameters_to_prune = []
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Linear, torch.nn.Conv2d)):
        parameters_to_prune.append((module, 'weight'))

# Perform pruning at different percentages: 10%, 20%, 30%, 40%, 50%, 60%, 70%, 80%
pruning_percentages = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

for prune_amount in pruning_percentages:
    # Apply global unstructured pruning (L1 norm - smallest magnitude weights)
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=prune_amount,
    )
    
    print(f"Pruning {int(prune_amount * 100)}% of weights (smallest magnitude)")
    test_model(model, test_data, mode=mode)
    
    # Calculate actual sparsity
    total_params = 0
    zero_params = 0
    for module, param_name in parameters_to_prune:
        param = getattr(module, param_name)
        total_params += param.numel()
        zero_params += (param == 0).sum().item()
    
    sparsity = 100. * zero_params / total_params
    print(f"Global sparsity: {sparsity:.2f}%")
    print("="*60 + "\n")
    
    # Remove pruning to continue to next percentage
    for module, param_name in parameters_to_prune:
        prune.remove(module, param_name)


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

## Inline Question 1:

What happened with the accuracy as the % of pruning increased ?
Why was that the case?


## Answer: 

The accuracy stayed pretty stable at first with low pruning percentages but started dropping significantly once we got past 50-60% pruning. This happened because the smaller weights we removed early on weren't that important, but eventually we started cutting weights that actually mattered for making correct predictions.

## Structured Pruning

Structured Pruning consists of removing a bigger chunk of the network parameters at the same time. Instead of removing only a few weights, it is commonplace to remove entire neurons. 

For example, in Convolutional Layers, removing filters can be beneficial to improve performance as it greatly decreases the amount of computation performed. However, some of these changes may affect output dimensions which may be carried over to other parts of the network. Therefore, when performing structured pruning one must always be aware of which parameters are going to be affected.

Using the previously trained model in the CINIC-10, perform Structured Prunning only in the Convolution layers of the DNN.

In [ ]:
################################################################################
# TODO: Perform unstrucuted Pruning over the trained model using 3 different  
# prunning percentages.                                
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
import torch
import torch.nn.utils.prune as prune

print("Model accuracy before structured pruning:")
test_model(model, test_data, mode=mode)
print("\n" + "="*60 + "\n")

conv_layers_to_prune = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        conv_layers_to_prune.append((module, 'weight'))
        print(f"Found Conv2d layer: {name}, shape: {module.weight.shape}")

print(f"\nTotal Conv2d layers to prune: {len(conv_layers_to_prune)}")
print("="*60 + "\n")

pruning_percentages = [0.1, 0.3, 0.5]

for prune_amount in pruning_percentages:
    # Prune entire filters (dim=0) based on L1 norm
    for module, param_name in conv_layers_to_prune:
        prune.ln_structured(
            module,
            name=param_name,
            amount=prune_amount,
            n=1,  
            dim=0  
        )
    
    print(f"Structured Pruning: {int(prune_amount * 100)}% of filters in Conv layers")
    test_model(model, test_data, mode=mode)
    
    total_params = 0
    zero_params = 0
    for module, param_name in conv_layers_to_prune:
        param = getattr(module, param_name)
        total_params += param.numel()
        zero_params += (param == 0).sum().item()
    
    sparsity = 100. * zero_params / total_params
    print(f"Conv layers sparsity: {sparsity:.2f}%")
    print("="*60 + "\n")
    
    for module, param_name in conv_layers_to_prune:
        prune.remove(module, param_name)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

## Inline Question 2:

What is the difference between performing Structured Pruning vs Dropout ? 
Why would it be beneficial to perform both techniques when developing a Neural Network?


## Answer: 

Dropout randomly turns off neurons during training to prevent overfitting, while structured pruning permanently removes entire filters or neurons after training to make the model smaller and faster. Using both is good because dropout makes the network learn better during training, and then pruning shrinks it down for deployment without losing much accuracy since dropout already made it robust.